# EDA: Games Base Dataset

Exploratory data analysis on `games_base.parquet` to validate data ingestion from nflverse.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

%matplotlib inline

## Load Data

In [ ]:
df = pd.read_parquet('../data/interim/games_base.parquet')
print(f"Total games: {len(df):,}")
print(f"Date range: {df['game_date'].min()} to {df['game_date'].max()}")
print(f"Seasons: {df['season'].min()} - {df['season'].max()}")

In [ ]:
df.head(10)

In [ ]:
df.info()

## Null Checks

In [ ]:
print("Null counts by column:")
null_counts = df.isnull().sum()
print(null_counts[null_counts > 0])

print("\nNull percentage:")
null_pct = (df.isnull().sum() / len(df) * 100).round(2)
print(null_pct[null_pct > 0])

## Games Per Season

In [ ]:
games_per_season = df.groupby('season').size()
print(games_per_season)

In [ ]:
fig, ax = plt.subplots(figsize=(14, 6))
games_per_season.plot(kind='bar', ax=ax, color='steelblue')
ax.set_title('Games Per Season (2000-2024)', fontsize=14, fontweight='bold')
ax.set_xlabel('Season', fontsize=12)
ax.set_ylabel('Number of Games', fontsize=12)
ax.axhline(y=267, color='red', linestyle='--', alpha=0.5, label='Expected (16-game season)')
ax.axhline(y=285, color='orange', linestyle='--', alpha=0.5, label='Expected (17-game season)')
ax.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print("\nNotes:")
print("- 2020 had 269 games (COVID-19 impact)")
print("- 2021+ have ~285 games (17-game regular season expansion)")
print("- Pre-2021 expected: ~267 games (256 regular + 11 playoff)")

## Game Type Distribution

In [ ]:
game_type_counts = df['game_type'].value_counts().sort_index()
print("Games by type:")
print(game_type_counts)
print(f"\nTotal: {game_type_counts.sum()}")

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Bar chart
game_type_counts.plot(kind='bar', ax=ax1, color='teal')
ax1.set_title('Games by Type', fontsize=14, fontweight='bold')
ax1.set_xlabel('Game Type', fontsize=12)
ax1.set_ylabel('Count', fontsize=12)
ax1.tick_params(axis='x', rotation=0)

# Pie chart (excluding REG for better visibility)
playoff_counts = game_type_counts[game_type_counts.index != 'REG']
ax2.pie(playoff_counts, labels=playoff_counts.index, autopct='%1.1f%%', startangle=90)
ax2.set_title('Playoff Games Distribution', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

print(f"\nExpected playoff games per season: 11 (4 WC + 4 DIV + 2 CONF + 1 SB)")
print(f"Seasons covered: {df['season'].nunique()}")
print(f"Expected total playoff games: {df['season'].nunique() * 11} = {11 * 25}")
print(f"Actual playoff games: {game_type_counts[game_type_counts.index != 'REG'].sum()}")

## Week Distribution

In [ ]:
print("Week range:", df['week'].min(), "-", df['week'].max())
print("\nGames by week:")
print(df['week'].value_counts().sort_index())

In [ ]:
# Check for weird weeks by season
week_by_season = df.groupby(['season', 'game_type'])['week'].agg(['min', 'max'])
print("Week range by season and game type:")
print(week_by_season.tail(10))

# Check if any REG games have week > 18
weird_weeks_reg = df[(df['game_type'] == 'REG') & (df['week'] > 18)]
print(f"\nREG games with week > 18: {len(weird_weeks_reg)}")
if len(weird_weeks_reg) > 0:
    print(weird_weeks_reg[['season', 'week', 'game_type', 'home_team', 'away_team']])

## Home Win Rate

In [ ]:
# Calculate home wins
df['home_win'] = (df['home_points'] > df['away_points']).astype(int)
df['tie'] = (df['home_points'] == df['away_points']).astype(int)

overall_home_win_rate = df['home_win'].mean()
overall_tie_rate = df['tie'].mean()

print(f"Overall home win rate: {overall_home_win_rate:.3f} ({overall_home_win_rate*100:.1f}%)")
print(f"Overall tie rate: {overall_tie_rate:.3f} ({overall_tie_rate*100:.1f}%)")
print(f"\nTotal ties: {df['tie'].sum()}")

In [ ]:
# Home win rate by season
home_win_by_season = df.groupby('season')['home_win'].mean()

fig, ax = plt.subplots(figsize=(14, 6))
home_win_by_season.plot(ax=ax, marker='o', linewidth=2, markersize=6, color='darkgreen')
ax.axhline(y=overall_home_win_rate, color='red', linestyle='--', alpha=0.7, 
           label=f'Overall: {overall_home_win_rate:.3f}')
ax.set_title('Home Win Rate by Season', fontsize=14, fontweight='bold')
ax.set_xlabel('Season', fontsize=12)
ax.set_ylabel('Home Win Rate', fontsize=12)
ax.set_ylim(0.45, 0.65)
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\nMin home win rate: {home_win_by_season.min():.3f} (Season {home_win_by_season.idxmin()})")
print(f"Max home win rate: {home_win_by_season.max():.3f} (Season {home_win_by_season.idxmax()})")

In [ ]:
# Home win rate by game type
home_win_by_type = df.groupby('game_type')['home_win'].mean().sort_values(ascending=False)

fig, ax = plt.subplots(figsize=(10, 5))
home_win_by_type.plot(kind='bar', ax=ax, color='forestgreen')
ax.set_title('Home Win Rate by Game Type', fontsize=14, fontweight='bold')
ax.set_xlabel('Game Type', fontsize=12)
ax.set_ylabel('Home Win Rate', fontsize=12)
ax.tick_params(axis='x', rotation=0)
ax.axhline(y=overall_home_win_rate, color='red', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

print("Home win rate by game type:")
print(home_win_by_type)

## Points Distribution

In [ ]:
# Summary statistics
df['total_points'] = df['home_points'] + df['away_points']
df['point_diff'] = df['home_points'] - df['away_points']

print("Points summary statistics:")
print(df[['home_points', 'away_points', 'total_points', 'point_diff']].describe())

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Home points
axes[0, 0].hist(df['home_points'], bins=50, color='blue', alpha=0.7, edgecolor='black')
axes[0, 0].axvline(df['home_points'].mean(), color='red', linestyle='--', 
                    label=f'Mean: {df["home_points"].mean():.1f}')
axes[0, 0].set_title('Home Points Distribution', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('Points')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].legend()

# Away points
axes[0, 1].hist(df['away_points'], bins=50, color='orange', alpha=0.7, edgecolor='black')
axes[0, 1].axvline(df['away_points'].mean(), color='red', linestyle='--', 
                    label=f'Mean: {df["away_points"].mean():.1f}')
axes[0, 1].set_title('Away Points Distribution', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('Points')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].legend()

# Total points
axes[1, 0].hist(df['total_points'], bins=50, color='green', alpha=0.7, edgecolor='black')
axes[1, 0].axvline(df['total_points'].mean(), color='red', linestyle='--', 
                    label=f'Mean: {df["total_points"].mean():.1f}')
axes[1, 0].set_title('Total Points Distribution', fontsize=12, fontweight='bold')
axes[1, 0].set_xlabel('Points')
axes[1, 0].set_ylabel('Frequency')
axes[1, 0].legend()

# Point differential (margin)
axes[1, 1].hist(df['point_diff'], bins=50, color='purple', alpha=0.7, edgecolor='black')
axes[1, 1].axvline(df['point_diff'].mean(), color='red', linestyle='--', 
                    label=f'Mean: {df["point_diff"].mean():.1f}')
axes[1, 1].axvline(0, color='black', linestyle='-', linewidth=2, alpha=0.5)
axes[1, 1].set_title('Point Differential Distribution (Home - Away)', fontsize=12, fontweight='bold')
axes[1, 1].set_xlabel('Point Differential')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].legend()

plt.tight_layout()
plt.show()

In [ ]:
# Extreme games
print("Highest scoring games:")
print(df.nlargest(5, 'total_points')[['game_date', 'home_team', 'away_team', 
                                        'home_points', 'away_points', 'total_points']])

print("\nLowest scoring games:")
print(df.nsmallest(5, 'total_points')[['game_date', 'home_team', 'away_team', 
                                        'home_points', 'away_points', 'total_points']])

print("\nBiggest blowouts:")
print(df.nlargest(5, 'point_diff')[['game_date', 'home_team', 'away_team', 
                                     'home_points', 'away_points', 'point_diff']])

## Team Analysis

In [ ]:
# Unique teams
home_teams = set(df['home_team'].unique())
away_teams = set(df['away_team'].unique())
all_teams = home_teams | away_teams

print(f"Total unique teams: {len(all_teams)}")
print(f"Teams: {sorted(all_teams)}")

# Check for historical team codes
historical_teams = ['SD', 'STL', 'OAK']  # San Diego, St. Louis, Oakland
current_teams = ['LAC', 'LA', 'LV']  # Los Angeles Chargers, Rams, Las Vegas

print("\nHistorical team codes found:")
for team in historical_teams:
    if team in all_teams:
        count = (df['home_team'] == team).sum() + (df['away_team'] == team).sum()
        seasons = df[(df['home_team'] == team) | (df['away_team'] == team)]['season'].unique()
        print(f"  {team}: {count} games, seasons {seasons.min()}-{seasons.max()}")

In [ ]:
# Games per team
team_games = pd.concat([df['home_team'], df['away_team']]).value_counts().sort_index()

fig, ax = plt.subplots(figsize=(14, 6))
team_games.plot(kind='bar', ax=ax, color='navy', alpha=0.7)
ax.set_title('Total Games by Team (2000-2024)', fontsize=14, fontweight='bold')
ax.set_xlabel('Team', fontsize=12)
ax.set_ylabel('Number of Games', fontsize=12)
ax.tick_params(axis='x', rotation=45)
plt.tight_layout()
plt.show()

print(f"\nGames per team (top 10):")
print(team_games.head(10))

## Data Quality Checks

In [ ]:
print("Data Quality Checks:\n")

# 1. Check for duplicate game_ids
duplicates = df[df.duplicated(subset=['game_id'], keep=False)]
print(f"1. Duplicate game_ids: {len(duplicates)}")
if len(duplicates) > 0:
    print(duplicates[['game_id', 'season', 'week', 'home_team', 'away_team']])

# 2. Check for negative points
negative_points = df[(df['home_points'] < 0) | (df['away_points'] < 0)]
print(f"\n2. Games with negative points: {len(negative_points)}")

# 3. Check for extremely high scores (> 70)
extreme_scores = df[(df['home_points'] > 70) | (df['away_points'] > 70)]
print(f"\n3. Games with scores > 70: {len(extreme_scores)}")
if len(extreme_scores) > 0:
    print(extreme_scores[['game_date', 'home_team', 'away_team', 'home_points', 'away_points']])

# 4. Check for team playing itself
self_play = df[df['home_team'] == df['away_team']]
print(f"\n4. Games where team plays itself: {len(self_play)}")

# 5. Check date consistency
date_issues = df[df['game_date'].isna()]
print(f"\n5. Games with null dates: {len(date_issues)}")

# 6. Check for future dates
future_games = df[df['game_date'] > pd.Timestamp.now()]
print(f"\n6. Games in the future: {len(future_games)}")
if len(future_games) > 0:
    print(future_games[['game_id', 'game_date', 'home_team', 'away_team', 'home_points', 'away_points']].head())

print("\n" + "="*60)
if (len(duplicates) == 0 and len(negative_points) == 0 and len(self_play) == 0 and len(date_issues) == 0):
    print("✓ All critical data quality checks PASSED")
else:
    print("⚠ Some data quality issues detected - review above")

## Summary

### Key Findings:
- Dataset contains games from 2000-2024 seasons
- Home field advantage is real (~57% win rate historically)
- Average total points per game: ~45 points
- Data quality looks good with expected nulls only in optional fields
- Historical team relocations (SD→LAC, STL→LA, OAK→LV) are present

### Next Steps:
1. Build rolling statistics features
2. Implement Elo rating system
3. Add rest days and scheduling features
4. Create baseline models